##Data Cleaning and Wrangling

In this notebook, we will examine the dataset for any empty fields and missing data, then drop unnecessary columns and fix the missing fields 

In [110]:
#Import necessary packages and display parameters

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import us
from datetime import datetime
from pytz import timezone
import pytz
import time
import operator

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)


In [111]:
df_merged = pd.read_csv('merged_trim_dataset.csv', index_col=False)
df_merged = df_merged.drop('Unnamed: 0', axis=1)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (73,74,76,80) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [112]:
df_merged.shape

(306279, 92)

In [113]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
0,2017,3,7,1,6,2017-07-01,WN,19393,WN,N8607M,4197,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,600,601.0,1.0,1.0,0.0,0.0,0600-0659,11.0,612.0,711.0,9.0,725,720.0,-5.0,0.0,0.0,-1.0,0700-0759,0.0,NaN,0.0,85.0,79.0,59.0,1.0,404.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,881.0,2017-07-01 10,ATL,2017-07-01 10:52,-84.4418,33.6301,71.1,70,96.32,250.00,4.00,0.0,30.09,1018.20,7,M,OVC,M,M,M,400.00,M,M,M,M,KATL 011052Z 25004KT 7SM OVC004 22/21 A3009 RMK AO2 SLP182 BINOVC T02170211 $,2017-07-01 10
1,2017,3,7,1,6,2017-07-01,OO,20304,OO,N187PQ,4930,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91,1330,1326.0,-4.0,0.0,0.0,-1.0,1300-1359,18.0,1344.0,1427.0,6.0,1447,1433.0,-14.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,77.0,67.0,43.0,1.0,226.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,382.0,2017-07-01 20,LAS,2017-07-01 20:56,-115.1522,36.0801,108.0,14,3.45,M,M,0.0,29.77,1004.70,10,M,CLR,M,M,M,M,M,M,M,M,KLAS 012056Z 10SM CLR 42/M10 A2977 RMK AO2 SLP047 T04221100 58021,2017-07-01 20
2,2017,3,7,1,6,2017-07-01,OO,20304,OO,N187PQ,4931,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91,1715,1715.0,0.0,0.0,0.0,0.0,1700-1759,17.0,1732.0,1813.0,4.0,1825,1817.0,-8.0,0.0,0.0,-1.0,1800-1859,0.0,NaN,0.0,70.0,62.0,41.0,1.0,226.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,382.0,2017-07-02 00,LAS,2017-07-02 00:56,-115.1522,36.0801,107.1,18,4.21,150.00,11.00,0.0,29.71,1002.80,10,21.00,FEW,M,M,M,20000.00,M,M,M,M,KLAS 020056Z 15011G21KT 10SM FEW200 42/M08 A2971 RMK AO2 PK WND 15027/0021 SLP028 T04171078,2017-07-02 00
3,2017,3,7,1,6,2017-07-01,OO,20304,OO,N250SY,4938,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,1305,1301.0,-4.0,0.0,0.0,-1.0,1300-1359,21.0,1322.0,1612.0,25.0,1637,1637.0,0.0,0.0,0.0,0.0,1600-1659,0.0,NaN,0.0,152.0,156.0,110.0,1.0,862.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,549.0,2017-07-01 20,LAX,2017-07-01 20:53,-118.3865,33.9382,71.1,61,70.42,260.00,9.00,0.0,29.88,1011.80,10,M,SCT,SCT,M,M,1900.00,2500.00,M,M,M,KLAX 012053Z 26009KT 10SM SCT019 SCT025 22/16 A2988 RMK AO2 SLP118 T02170161 58001,2017-07-01 20
4,2017,3,7,1,6,2017-07-01,OO,20304,OO,N253SY,4940,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,10713,1071302,30713,BOI,"Boise, ID",ID,16,Idaho,83,1850,1844.0,-6.0,0.0,0.0,-1.0,1800-1859,13.0,1857.0,2054.0,4.0,2123,2058.0,-25.0,0.0,0.0,-2.0,2100-2159,0.0,NaN,0.0,93.0,74.0,57.0,1.0,399.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,402.0,2017-07-02 01,SEA,2017-07-02 01:53,-122.3144,47.4447,71.1,53.1,52.96,0.00,0.00,0.0,30.04,1017.80,10,M,CLR,M,M,M,M,M,M,M,M,KSEA 020153Z 00000KT 10SM CLR 22/12 A3004 RMK AO2 SLP178 T02170117,2017-07-02 01


In [114]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306279 entries, 0 to 306278
Data columns (total 92 columns):
Year                    306279 non-null int64
Quarter                 306279 non-null int64
Month                   306279 non-null int64
DayofMonth              306279 non-null int64
DayOfWeek               306279 non-null int64
FlightDate              306279 non-null object
UniqueCarrier           306279 non-null object
AirlineID               306279 non-null int64
Carrier                 306279 non-null object
TailNum                 305751 non-null object
FlightNum               306279 non-null int64
OriginAirportID         306279 non-null int64
OriginAirportSeqID      306279 non-null int64
OriginCityMarketID      306279 non-null int64
Origin                  306279 non-null object
OriginCityName          306279 non-null object
OriginState             306279 non-null object
OriginStateFips         306279 non-null int64
OriginStateName         306279 non-null object
OriginW

In [115]:
#Check percentage of empty fields in each column

frac_nulls = {col: (sum(df_merged[col].isnull()/len(df_merged))) for col in df_merged.columns}
frac_nulls = sorted(frac_nulls.items(), key = operator.itemgetter(1))
frac_nulls

[('Year', 0.0),
 ('Quarter', 0.0),
 ('Month', 0.0),
 ('DayofMonth', 0.0),
 ('DayOfWeek', 0.0),
 ('FlightDate', 0.0),
 ('UniqueCarrier', 0.0),
 ('AirlineID', 0.0),
 ('Carrier', 0.0),
 ('FlightNum', 0.0),
 ('OriginAirportID', 0.0),
 ('OriginAirportSeqID', 0.0),
 ('OriginCityMarketID', 0.0),
 ('Origin', 0.0),
 ('OriginCityName', 0.0),
 ('OriginState', 0.0),
 ('OriginStateFips', 0.0),
 ('OriginStateName', 0.0),
 ('OriginWac', 0.0),
 ('DestAirportID', 0.0),
 ('DestAirportSeqID', 0.0),
 ('DestCityMarketID', 0.0),
 ('Dest', 0.0),
 ('DestCityName', 0.0),
 ('DestState', 0.0),
 ('DestStateFips', 0.0),
 ('DestStateName', 0.0),
 ('DestWac', 0.0),
 ('CRSDepTime', 0.0),
 ('DepTimeBlk', 0.0),
 ('CRSArrTime', 0.0),
 ('ArrTimeBlk', 0.0),
 ('Cancelled', 0.0),
 ('Diverted', 0.0),
 ('CRSElapsedTime', 0.0),
 ('Flights', 0.0),
 ('Distance', 0.0),
 ('DistanceGroup', 0.0),
 ('IsDelay', 0.0),
 ('AptPreviousDateDelay', 0.0),
 ('AptNoOfFlight', 0.0),
 ('UTCFlightDateTime', 0.0),
 ('station', 0.0),
 ('valid', 0.0

In [116]:
(df_merged.Cancelled.sum())/len(df_merged)

0.014747991210628218

NaN in DepDelay and other fields are due to flight cancelled, therefore removing all cancelled flights will removed most of the NaN in those columns.

NaN in ArrDelay and other related fieds are due to flight diverted, and we are interested only in departure delay. Diverted flight will still be included. These columns will be dropped due to unrelated to our prediction.

In [117]:
df_merged = df_merged[~(df_merged.Cancelled == 1)]


In [118]:
%%time

df_merged = df_merged[~pd.isnull(df_merged.TailNum)]

CPU times: user 250 ms, sys: 217 ms, total: 467 ms
Wall time: 613 ms


In [119]:
df_merged.shape

(301762, 92)

In [120]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
0,2017,3,7,1,6,2017-07-01,WN,19393,WN,N8607M,4197,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,600,601.0,1.0,1.0,0.0,0.0,0600-0659,11.0,612.0,711.0,9.0,725,720.0,-5.0,0.0,0.0,-1.0,0700-0759,0.0,NaN,0.0,85.0,79.0,59.0,1.0,404.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,881.0,2017-07-01 10,ATL,2017-07-01 10:52,-84.4418,33.6301,71.1,70,96.32,250.00,4.00,0.0,30.09,1018.20,7,M,OVC,M,M,M,400.00,M,M,M,M,KATL 011052Z 25004KT 7SM OVC004 22/21 A3009 RMK AO2 SLP182 BINOVC T02170211 $,2017-07-01 10
1,2017,3,7,1,6,2017-07-01,OO,20304,OO,N187PQ,4930,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91,1330,1326.0,-4.0,0.0,0.0,-1.0,1300-1359,18.0,1344.0,1427.0,6.0,1447,1433.0,-14.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,77.0,67.0,43.0,1.0,226.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,382.0,2017-07-01 20,LAS,2017-07-01 20:56,-115.1522,36.0801,108.0,14,3.45,M,M,0.0,29.77,1004.70,10,M,CLR,M,M,M,M,M,M,M,M,KLAS 012056Z 10SM CLR 42/M10 A2977 RMK AO2 SLP047 T04221100 58021,2017-07-01 20
2,2017,3,7,1,6,2017-07-01,OO,20304,OO,N187PQ,4931,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91,1715,1715.0,0.0,0.0,0.0,0.0,1700-1759,17.0,1732.0,1813.0,4.0,1825,1817.0,-8.0,0.0,0.0,-1.0,1800-1859,0.0,NaN,0.0,70.0,62.0,41.0,1.0,226.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,382.0,2017-07-02 00,LAS,2017-07-02 00:56,-115.1522,36.0801,107.1,18,4.21,150.00,11.00,0.0,29.71,1002.80,10,21.00,FEW,M,M,M,20000.00,M,M,M,M,KLAS 020056Z 15011G21KT 10SM FEW200 42/M08 A2971 RMK AO2 PK WND 15027/0021 SLP028 T04171078,2017-07-02 00
3,2017,3,7,1,6,2017-07-01,OO,20304,OO,N250SY,4938,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,1305,1301.0,-4.0,0.0,0.0,-1.0,1300-1359,21.0,1322.0,1612.0,25.0,1637,1637.0,0.0,0.0,0.0,0.0,1600-1659,0.0,NaN,0.0,152.0,156.0,110.0,1.0,862.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,549.0,2017-07-01 20,LAX,2017-07-01 20:53,-118.3865,33.9382,71.1,61,70.42,260.00,9.00,0.0,29.88,1011.80,10,M,SCT,SCT,M,M,1900.00,2500.00,M,M,M,KLAX 012053Z 26009KT 10SM SCT019 SCT025 22/16 A2988 RMK AO2 SLP118 T02170161 58001,2017-07-01 20
4,2017,3,7,1,6,2017-07-01,OO,20304,OO,N253SY,4940,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,10713,1071302,30713,BOI,"Boise, ID",ID,16,Idaho,83,1850,1844.0,-6.0,0.0,0.0,-1.0,1800-1859,13.0,1857.0,2054.0,4.0,2123,2058.0,-25.0,0.0,0.0,-2.0,2100-2159,0.0,NaN,0.0,93.0,74.0,57.0,1.0,399.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,402.0,2017-07-02 01,SEA,2017-07-02 01:53,-122.3144,47.4447,71.1,53.1,52.96,0.00,0.00,0.0,30.04,1017.80,10,M,CLR,M,M,M,M,M,M,M,M,KSEA 020153Z 00000KT 10SM CLR 22/12 A3004 RMK AO2 SLP178 T02170117,2017-07-02 01


In [121]:
df_merged.FlightNum = df_merged.Carrier + df_merged.FlightNum.astype(str)

In [122]:
df_merged.IsDelay.sum()/len(df_merged)

0.2612423035372247

In [123]:
#By examining the raw METAR both columns ' drct' and ' sknt' are missing are because those are not reported in the METAR

df_merged[(df_merged[' drct'] == 'M') & (df_merged[' sknt'] == 'M')][' metar']

1                     KLAS 012056Z 10SM CLR 42/M10 A2977 RMK AO2 SLP047 T04221100 58021
291       KPHX 012351Z 10SM CLR 43/M02 A2971 RMK AO2 SLP037 T04331017 10444 20372 56020
369                KDEN 011953Z 10SM FEW100 FEW220 29/00 A3011 RMK AO2 SLP114 T02940000
593                   KLAS 012056Z 10SM CLR 42/M10 A2977 RMK AO2 SLP047 T04221100 58021
1080      KPHX 012351Z 10SM CLR 43/M02 A2971 RMK AO2 SLP037 T04331017 10444 20372 56020
1095      KPHX 012351Z 10SM CLR 43/M02 A2971 RMK AO2 SLP037 T04331017 10444 20372 56020
1246                  KLAS 012056Z 10SM CLR 42/M10 A2977 RMK AO2 SLP047 T04221100 58021
1486      KPHX 012351Z 10SM CLR 43/M02 A2971 RMK AO2 SLP037 T04331017 10444 20372 56020
1631               KDEN 011953Z 10SM FEW100 FEW220 29/00 A3011 RMK AO2 SLP114 T02940000
1655               KDEN 011953Z 10SM FEW100 FEW220 29/00 A3011 RMK AO2 SLP114 T02940000
1681               KDEN 011953Z 10SM FEW100 FEW220 29/00 A3011 RMK AO2 SLP114 T02940000
2509                  KLAS 01205

In [124]:
df_merged[df_merged[' metar'].str.contains("00000KT")]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
4,2017,3,7,1,6,2017-07-01,OO,20304,OO,N253SY,OO4940,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,10713,1071302,30713,BOI,"Boise, ID",ID,16,Idaho,83,1850,1844.0,-6.0,0.0,0.0,-1.0,1800-1859,13.0,1857.0,2054.0,4.0,2123,2058.0,-25.0,0.0,0.0,-2.0,2100-2159,0.0,NaN,0.0,93.0,74.0,57.0,1.0,399.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,402.0,2017-07-02 01,SEA,2017-07-02 01:53,-122.3144,47.4447,71.1,53.1,52.96,0.00,0.00,0.00,30.04,1017.80,10,M,CLR,M,M,M,M,M,M,M,M,KSEA 020153Z 00000KT 10SM CLR 22/12 A3004 RMK AO2 SLP178 T02170117,2017-07-02 01
17,2017,3,7,1,6,2017-07-01,OO,20304,OO,N418SW,OO4981,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,11778,1177801,31778,FSM,"Fort Smith, AR",AR,5,Arkansas,71,2139,2322.0,103.0,103.0,1.0,6.0,2100-2159,12.0,2334.0,2355.0,5.0,2228,2400.0,92.0,92.0,1.0,6.0,2200-2259,0.0,NaN,0.0,109.0,98.0,81.0,1.0,579.0,3,0.0,0.0,0.0,0.0,92.0,NaN,NaN,1,0.0,881.0,2017-07-02 01,ATL,2017-07-02 01:52,-84.4418,33.6301,75.0,69.1,81.9,0.00,0.00,0.00,30.14,1019.70,10,M,FEW,FEW,SCT,BKN,3400.00,5500.00,9000.00,20000.00,TS,KATL 020152Z 00000KT 10SM TS FEW034CB FEW055 SCT090 BKN200 24/21 A3014 RMK AO2 RAE00 TSE25B52 SLP197 FRQ LTGICCGCA VC SW-NW TS VC SW-NW MOV SE P0000 T02390206 $,2017-07-02 01
55,2017,3,7,1,6,2017-07-01,OO,20304,OO,N633SK,OO4878,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,11603,1160302,31603,EUG,"Eugene, OR",OR,41,Oregon,92,1840,1835.0,-5.0,0.0,0.0,-1.0,1800-1859,11.0,1846.0,1925.0,2.0,1950,1927.0,-23.0,0.0,0.0,-2.0,1900-1959,0.0,NaN,0.0,70.0,52.0,39.0,1.0,234.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,402.0,2017-07-02 01,SEA,2017-07-02 01:53,-122.3144,47.4447,71.1,53.1,52.96,0.00,0.00,0.00,30.04,1017.80,10,M,CLR,M,M,M,M,M,M,M,M,KSEA 020153Z 00000KT 10SM CLR 22/12 A3004 RMK AO2 SLP178 T02170117,2017-07-02 01
113,2017,3,7,1,6,2017-07-01,OO,20304,OO,N255SY,OO4852,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,14679,1467903,33570,SAN,"San Diego, CA",CA,6,California,91,1850,1846.0,-4.0,0.0,0.0,-1.0,1800-1859,12.0,1858.0,2103.0,4.0,2145,2107.0,-38.0,0.0,0.0,-2.0,2100-2159,0.0,NaN,0.0,175.0,141.0,125.0,1.0,1050.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,402.0,2017-07-02 01,SEA,2017-07-02 01:53,-122.3144,47.4447,71.1,53.1,52.96,0.00,0.00,0.00,30.04,1017.80,10,M,CLR,M,M,M,M,M,M,M,M,KSEA 020153Z 00000KT 10SM CLR 22/12 A3004 RMK AO2 SLP178 T02170117,2017-07-02 01
130,2017,3,7,1,6,2017-07-01,OO,20304,OO,N613SK,OO4693,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,1830,1838.0,8.0,8.0,0.0,0.0,1800-1859,12.0,1850.0,2119.0,10.0,2130,2129.0,-1.0,0.0,0.0,-1.0,2100-2159,0.0,NaN,0.0,120.0,111.0,89.0,1.0,689.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,402.0,2017-07-02 01,SEA,2017-07-02 01:53,-122.3144,47.4447,71.1,53.1,52.96,0.00,0.00,0.00,30.04,1017.80,10,M,CLR,M,M,M,M,M,M,M,M,KSEA 020153Z 00000KT 10SM CLR 22/12 A3004 RMK AO2 SLP178 T02170117,2017-07-02 01
135,2017,3,7,1,6,2017-07-01,OO,20304,OO,N765SK,OO4702,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,14828,1482803,34828,SIT,"Sitka, AK",A

If wind is calm, the code will be 00000KT instead of just missing.

In [125]:
#Remove rows that both wind direction and speed are missing.

df_merged = df_merged[~((df_merged[' drct'] == 'M') & (df_merged[' sknt'] == 'M'))]


In [126]:
#Check no more Missing in both wind direction and speed
df_merged[((df_merged[' drct'] == 'M') & (df_merged[' sknt'] == 'M'))]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr


In [127]:
#Check for rows that ' drct' is having value 'M' that is VRB wind, we will later change it to 999.

df_merged[df_merged[' drct'] == 'M']

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
42,2017,3,7,1,6,2017-07-01,OO,20304,OO,N259SY,OO4926,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,14771,1477102,32457,SFO,"San Francisco, CA",CA,6,California,91,2120,2118.0,-2.0,0.0,0.0,-1.0,2100-2159,17.0,2135.0,2230.0,14.0,2251,2244.0,-7.0,0.0,0.0,-1.0,2200-2259,0.0,NaN,0.0,91.0,86.0,55.0,1.0,337.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,549.0,2017-07-02 04,LAX,2017-07-02 04:53,-118.3865,33.9382,64.9,60.1,84.44,M,3.00,0.0,29.92,1012.80,10,M,OVC,M,M,M,1300.00,M,M,M,M,KLAX 020453Z VRB03KT 10SM OVC013 18/16 A2992 RMK AO2 SLP128 T01830156,2017-07-02 04
89,2017,3,7,1,6,2017-07-01,OO,20304,OO,N924SW,OO5021,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,13476,1347603,34922,MRY,"Monterey, CA",CA,6,California,91,2150,2150.0,0.0,0.0,0.0,0.0,2100-2159,16.0,2206.0,2253.0,5.0,2304,2258.0,-6.0,0.0,0.0,-1.0,2300-2359,0.0,NaN,0.0,74.0,68.0,47.0,1.0,266.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,549.0,2017-07-02 04,LAX,2017-07-02 04:53,-118.3865,33.9382,64.9,60.1,84.44,M,3.00,0.0,29.92,1012.80,10,M,OVC,M,M,M,1300.00,M,M,M,M,KLAX 020453Z VRB03KT 10SM OVC013 18/16 A2992 RMK AO2 SLP128 T01830156,2017-07-02 04
96,2017,3,7,1,6,2017-07-01,OO,20304,OO,N956SW,OO5032,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,14794,1479403,34794,SGU,"St. George, UT",UT,49,Utah,87,1125,1120.0,-5.0,0.0,0.0,-1.0,1100-1159,27.0,1147.0,1305.0,6.0,1310,1311.0,1.0,1.0,0.0,0.0,1300-1359,0.0,NaN,0.0,105.0,111.0,78.0,1.0,517.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,609.0,2017-07-01 17,DEN,2017-07-01 17:53,-104.6575,39.8328,80.1,36,20.44,M,3.00,0.0,30.15,1013.90,10,M,FEW,FEW,M,M,9000.00,22000.00,M,M,M,KDEN 011753Z VRB03KT 10SM FEW090 FEW220 27/02 A3015 RMK AO2 SLP139 T02670022 10267 20117 58013,2017-07-01 17
156,2017,3,7,1,6,2017-07-01,OO,20304,OO,N247SY,OO4639,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,1220,1217.0,-3.0,0.0,0.0,-1.0,1200-1259,13.0,1230.0,1353.0,9.0,1423,1402.0,-21.0,0.0,0.0,-2.0,1400-1459,0.0,NaN,0.0,183.0,165.0,143.0,1.0,1024.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,609.0,2017-07-01 18,DEN,2017-07-01 18:53,-104.6575,39.8328,82.9,30.9,15.21,M,6.00,0.0,30.13,1012.40,10,M,FEW,FEW,M,M,10000.00,22000.00,M,M,M,KDEN 011853Z VRB06KT 10SM FEW100 FEW220 28/M01 A3013 RMK AO2 SLP124 T02831006,2017-07-01 18
266,2017,3,7,1,6,2017-07-01,OO,20304,OO,N203SY,OO5435,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,1107,1139.0,32.0,32.0,1.0,2.0,1100-1159,12.0,1151.0,1247.0,6.0,1241,1253.0,12.0,12.0,0.0,0.0,1200-1259,0.0,NaN,0.0,94.0,74.0,56.0,1.0,370.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,391.0,2017-07-01 17,PHX,2017-07-01 17:51,-112.0116,33.4343,99.0,34,10.41,M,3.00,0.0,29.81,1007.20,10,M,CLR,M,M,M,M,M,M,M,M,KPHX 011751Z VRB03KT 10SM CLR 37/01 A2981 RMK AO2 SLP072 T03720011 10378 20267 58000,2017-07-01 17
268,2017,3,7,1,6,2017-07-01,OO,20304,OO,N151SY,OO5436,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,1818,1822.0,4.0,4

The model is for prediction of departure delay, any post flight information are irrelevant, therefore WheelsOn, TaxiIn, ArrTime, ActualElapsedTime, AirTime can be dropped.

In [128]:
%%time

df_merged.drop(['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',
                'CancellationCode','FirstDepTime','TotalAddGTime','UniqueCarrier','AirlineID',
                'OriginAirportID','OriginAirportSeqID','OriginCityMarketID','OriginStateFips',
                'OriginStateName','OriginCityName',
                'DestAirportID','DestAirportSeqID','DestCityMarketID','DestStateFips',
                'DestStateName','DestCityName',
                'DepTime','DepDelay','DepDelayMinutes','DepDel15','DepartureDelayGroups',
                'ArrDelay','ArrDelayMinutes','ArrDel15','ArrivalDelayGroups',
                'ArrTime','ActualElapsedTime','AirTime',
                'WheelsOn','TaxiIn',
                'Cancelled','Diverted','UTCFlightDateTime',
                'valid','station','lat','lon',' mslp','DateHr',' metar'], 
               axis=1, inplace=True)

CPU times: user 219 ms, sys: 353 ms, total: 572 ms
Wall time: 818 ms


In [129]:
df_merged.shape

(300440, 46)

In [130]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Carrier,TailNum,FlightNum,Origin,OriginState,OriginWac,Dest,DestState,DestWac,CRSDepTime,DepTimeBlk,TaxiOut,WheelsOff,CRSArrTime,ArrTimeBlk,CRSElapsedTime,Flights,Distance,DistanceGroup,IsDelay,AptPreviousDateDelay,AptNoOfFlight,tmpf,dwpf,relh,drct,sknt,p01i,alti,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx
0,2017,3,7,1,6,2017-07-01,WN,N8607M,WN4197,ATL,GA,34,MCO,FL,33,600,0600-0659,11.0,612.0,725,0700-0759,85.0,1.0,404.0,2,0,0.0,881.0,71.1,70,96.32,250.00,4.00,0.0,30.09,7,M,OVC,M,M,M,400.00,M,M,M,M
2,2017,3,7,1,6,2017-07-01,OO,N187PQ,OO4931,LAS,NV,85,SNA,CA,91,1715,1700-1759,17.0,1732.0,1825,1800-1859,70.0,1.0,226.0,1,0,0.0,382.0,107.1,18,4.21,150.00,11.00,0.0,29.71,10,21.00,FEW,M,M,M,20000.00,M,M,M,M
3,2017,3,7,1,6,2017-07-01,OO,N250SY,OO4938,LAX,CA,91,DEN,CO,82,1305,1300-1359,21.0,1322.0,1637,1600-1659,152.0,1.0,862.0,4,0,0.0,549.0,71.1,61,70.42,260.00,9.00,0.0,29.88,10,M,SCT,SCT,M,M,1900.00,2500.00,M,M,M
4,2017,3,7,1,6,2017-07-01,OO,N253SY,OO4940,SEA,WA,93,BOI,ID,83,1850,1800-1859,13.0,1857.0,2123,2100-2159,93.0,1.0,399.0,2,0,0.0,402.0,71.1,53.1,52.96,0.00,0.00,0.0,30.04,10,M,CLR,M,M,M,M,M,M,M,M
5,2017,3,7,1,6,2017-07-01,OO,N609SK,OO4941,SEA,WA,93,GEG,WA,93,2110,2100-2159,16.0,2143.0,2215,2200-2259,65.0,1.0,224.0,1,0,0.0,402.0,62.1,54,74.75,280.00,3.00,0.0,30.06,10,M,CLR,M,M,M,M,M,M,M,M


In [131]:
df_merged[' drct'] = df_merged[' drct'].replace({'M':999})
df_merged[' gust'] = df_merged[' gust'].replace({'M':0})
df_merged[' skyl1'] = df_merged[' skyl1'].replace({'M':99999})
df_merged[' skyl2'] = df_merged[' skyl2'].replace({'M':99999})
df_merged[' skyl3'] = df_merged[' skyl3'].replace({'M':99999})
df_merged[' skyl4'] = df_merged[' skyl4'].replace({'M':99999})

In [132]:
frac_nulls = {col: (sum(df_merged[col].isnull()/len(df_merged))) for col in df_merged.columns}
frac_nulls = sorted(frac_nulls.items(), key = operator.itemgetter(1))
frac_nulls

[('Year', 0.0),
 ('Quarter', 0.0),
 ('Month', 0.0),
 ('DayofMonth', 0.0),
 ('DayOfWeek', 0.0),
 ('FlightDate', 0.0),
 ('Carrier', 0.0),
 ('TailNum', 0.0),
 ('FlightNum', 0.0),
 ('Origin', 0.0),
 ('OriginState', 0.0),
 ('OriginWac', 0.0),
 ('Dest', 0.0),
 ('DestState', 0.0),
 ('DestWac', 0.0),
 ('CRSDepTime', 0.0),
 ('DepTimeBlk', 0.0),
 ('TaxiOut', 0.0),
 ('WheelsOff', 0.0),
 ('CRSArrTime', 0.0),
 ('ArrTimeBlk', 0.0),
 ('CRSElapsedTime', 0.0),
 ('Flights', 0.0),
 ('Distance', 0.0),
 ('DistanceGroup', 0.0),
 ('IsDelay', 0.0),
 ('AptPreviousDateDelay', 0.0),
 ('AptNoOfFlight', 0.0),
 ('tmpf', 0.0),
 (' dwpf', 0.0),
 (' relh', 0.0),
 (' drct', 0.0),
 (' sknt', 0.0),
 (' p01i', 0.0),
 (' alti', 0.0),
 (' vsby', 0.0),
 (' gust', 0.0),
 (' skyc1', 0.0),
 (' skyc2', 0.0),
 (' skyc3', 0.0),
 (' skyc4', 0.0),
 (' skyl1', 0.0),
 (' skyl2', 0.0),
 (' skyl3', 0.0),
 (' skyl4', 0.0),
 (' presentwx', 0.0)]

In [133]:
#Move IsDelay column which will be the target column to the back

cols = list(df_merged.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('IsDelay')) #Remove IsDelay from list
df_merged = df_merged[cols+['IsDelay']]

In [134]:
%%time

df_merged.to_csv('cleaned_dataset.csv')

CPU times: user 11 s, sys: 731 ms, total: 11.7 s
Wall time: 14.4 s
